In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0022726058959960938
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 35.1809375846147
200000 53.39632569515115
300000 56.278035730389824
400000 54.84600885152388
500000 53.84814762235533
600000 58.10165016263479
700000 54.019238417682715
800000 60.76876137490847
900000 56.55971773649077
1000000 42.99009285280783
1100000 38.55680884003555
1200000 48.46517572964322
1300000 52.268890026234246
1400000 55.35344314823703
1500000 65.17048004922333
1600000 66.14386236090806
1700000 66.65828362211592
1800000 57.48643975492894
1900000 57.28043296632765
2000000 35.3208241087341
2100000 28.36538479169581
2200000 58.669477982682956
2300000 55.9511103831512
2400000 54.42034420427684
2500000 62.34593609618452
2600000 51.01808105589793
2700000 45.109174561723584
2800000 53.80556369384371
2900000 52.29674895710165
3000000 40.895612340832

30000000 76.7563271560576
30100000 76.73234866220321
30200000 74.97152923120703
30300000 31.97602158996824
30400000 75.19376661285435
30500000 75.45633198513524
30600000 73.95807682656643
30700000 76.56360768465163
30800000 68.36945464061453
30900000 67.43108441560216
31000000 57.36148008714268
31100000 41.697332053045145
31200000 64.64097514710025
31300000 66.37767845092603
31400000 70.11380540705893
31500000 69.91901973600592
31600000 73.79737052684409
31700000 73.54767243883731
31800000 41.000358396029895
31900000 73.2289126693701
32000000 74.31173634152279
32100000 77.03560960745614
32200000 75.37957387696434
32300000 36.540231367007934
32400000 74.13945722591568
32500000 77.43091787634489
32600000 74.43476028021398
32700000 70.94077769856683
32800000 74.7371145275966
32900000 70.15523387989516
33000000 56.53586714260776
33100000 39.13862380187328
33200000 68.59326788629029
33300000 71.87086263347395
33400000 71.42672641568716
33500000 75.1678034904718
33600000 75.13965739282531
33

60400000 78.44708610853694
60500000 48.3400603008707
60600000 51.64634388878688
60700000 65.35464018401271
60800000 64.35530802107148
60900000 66.36052414222746
61000000 55.9052483955602
61100000 35.04602643646355
61200000 66.77414628251502
61300000 63.74305832760639
61400000 68.04027400708856
61500000 65.09712544098606
61600000 34.65650356401296
61700000 68.6517035213563
61800000 71.08682556885665
61900000 70.58202036358396
62000000 74.30226149639587
62100000 79.33165134559458
62200000 72.00988208196813
62300000 71.21120463451223
62400000 68.57372303772563
62500000 57.9946572504945
62600000 43.153232766610905
62700000 61.42338570231633
62800000 66.74194827572349
62900000 62.28168702440756
63000000 57.33317783508
63100000 34.77003284019502
63200000 62.60590670508976
63300000 67.26280904974571
63400000 61.964576835606834
63500000 61.49993416557719
63600000 39.25993240955487
63700000 65.74680165521274
63800000 74.92648192769126
63900000 69.63202035880698
64000000 74.66969176681
64100000 

90800000 27.226687414169543
90900000 60.3249484816491
91000000 53.00302932299216
91100000 32.472181427373805
91200000 56.20141485853109
91300000 49.13108095681898
91400000 43.4245076925404
91500000 57.052539759848294
91600000 64.01935896335958
91700000 60.351735604680385
91800000 64.43282697414166
91900000 70.72072574576926
92000000 65.52815126951165
92100000 70.16334361625432
92200000 68.88711241375827
92300000 70.43037772708651
92400000 66.18654274978546
92500000 62.64296466493567
92600000 62.100727994334996
92700000 61.47292630223148
92800000 30.005250256966743
92900000 61.84213335739696
93000000 55.3873958262907
93100000 31.082820649132408
93200000 58.68905858735101
93300000 44.070542125393935
93400000 52.647913672943965
93500000 61.64486552383606
93600000 60.03299814200812
93700000 63.33621917059866
93800000 66.06027287904888
93900000 66.84066786522762
94000000 67.32799049173587
94100000 65.68007764540768
94200000 66.42080023555818
94300000 63.81473089062043
94400000 63.6230789623

120200000 76.56535359507065
120300000 79.25557982633589
120400000 75.22804296483677
120500000 78.25355201077402
120600000 74.93092221370144
120700000 74.1310854338773
120800000 75.18172626056112
120900000 75.26822274381259
121000000 38.26101890438843
121100000 11.367731098995977
121200000 65.19003858012617
121300000 71.94052288533194
121400000 72.39808473446692
121500000 75.72462261574462
121600000 76.5210829010888
121700000 76.24884854149127
121800000 77.38842706216506
121900000 79.937307024513
122000000 78.57691246283119
122100000 25.71621357473888
122200000 6.315438925971673
122300000 6.7282527966811125
122400000 6.9401719279656255
122500000 6.647585404791159
122600000 8.454088336060924
122700000 6.77638105926163
122800000 6.822220856371378
122900000 6.3297950033691945
123000000 6.937640687892824
123100000 6.734591625683275
123200000 6.750763765662045
123300000 6.3798887197808085
123400000 7.081550169369431
123500000 6.6284961136860625
123600000 6.536520967147277
123700000 6.7767137

...

884500000 2.1369482787332843
884600000 3.33597339097866
884700000 2.4162133611374306
884800000 3.517130574028474
884900000 2.135786853050482
885000000 3.074828275798518
885100000 2.266681297456218
885200000 3.0143879803405342
885300000 3.0479124010756267
885400000 2.290862115681173
885500000 3.1733779212882043
885600000 2.0685209044734054
885700000 3.4229251030141907
885800000 1.697633975589498
885900000 3.2612841209018737
886000000 1.7677086246397262
886100000 3.032179066465199
886200000 2.635048738416871
886300000 2.5962809082040303
886400000 4.033388235652967
886500000 2.510903914296393
886600000 1.7138696868018137
886700000 0.5289587746800979
886800000 0.5319160814067179
886900000 0.5309261313881715
887000000 0.5296837223496258
887100000 0.5302702336617476
887200000 0.5262426496208767
887300000 0.5275598228926083
887400000 0.5267996201778643
887500000 0.521890600943354
887600000 0.528297896353362
887700000 2.7812801540473924
887800000 2.5600434504850362
887900000 4.240937004137349


909200000 0.35466192290697496
best so far: 0
type: [1, 3, 6, 7, 1] 126
cases of this type: 37044
best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 0.9896426589569925
909400000 0.4584921658507161
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.3481468866118133
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.33226408109140726
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.3226426850958374
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.6271125293899417
909900000 0.9042526766804709
910000000 0.898324871

920900000 0.8903501360547185
921000000 1.0612872666287878
921100000 1.3759949573997603
921200000 0.8281786500201583
921300000 1.2743766035924269
921400000 0.9343998116408341
921500000 1.1951314487784583
921600000 1.3543725944019707
921700000 0.9036454687533478
921800000 1.3528973733744263
921900000 1.0341728776022414
922000000 1.3030809492855668
922100000 0.39127399003559943
922200000 0.3254321002288878
922300000 0.26686395868839885
922400000 0.2902187583909174
922500000 0.247544789034609
922600000 0.24803580501737527
922700000 0.22940086376368207
922800000 0.23928962701603107
922900000 0.23863060139364534
923000000 0.22699689190084937
923100000 0.2340954425144818
923200000 0.24718408509905734
923300000 0.7370086116118272
923400000 1.0666711837430756
923500000 1.3618061400585302
923600000 1.2847345446748555
923700000 0.9476430766189687
923800000 1.2870995824150535
923900000 0.9030726114066542
924000000 1.0312993772979213
924100000 1.2249633517206733
924200000 0.7578627518613557
9243000

944700000 0.19049050529955228
best so far: 0
type: [2, 3, 1, 3, 7] 126
cases of this type: 194481
944800000 0.17353618603921864
944900000 0.11691489396549463
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.11741299294361672
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.10265033789122435
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.10391515587467749
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 1, 3] 126
cases of this type: 5103
best so far: 0
typ

955400000 0.008517006024563314
955500000 0.007156639925148752
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0
type: [6, 1, 1, 7, 3] 126
cases of this type: 27783
955600000 0.005054038223371242
best so far: 0
type: [6, 1, 1, 21, 1] 126
cases of this type: 9261
best so far: 0
type: [6, 1, 3, 1, 7] 126
cases of this type: 21609
best so far: 0
type: [6, 1, 3, 7, 1] 126
cases of this type: 3087
best so far: 0
type: [6, 1, 7, 1, 3] 126
cases of this type: 3969
best so far: 0
type: [6, 1, 7, 3, 1] 126
cases of this type: 1323
best so far: 0
type: [6, 1, 21, 1, 1] 126
cases of this type: 441
best so far: 0
type: [6, 3, 1, 1, 7] 126
cases of this type: 7203
best so far: 0
type: [6, 3, 1, 7, 1] 126
cases of this type: 1029
best so far: 0
type: [6, 3, 7, 1, 1] 126
cases of this type: 147
best so far: 0
type: [6, 7, 1, 1, 3] 126
cases of this type: 567
best so far: 0
type: [6, 7, 1, 3, 1] 126
cases of this type: 189
best so far: 0
type: [6, 7, 3, 1, 1] 126
cases o